### **Approach to the problem**

I used n-gram interpolation algorithm to enable the game to guess a letter. This method utilizes the statistical properties of language and linguistic patterns to estimate the likelihood of certain letters appearing in specific positions, leading to a more accurate and informed guesses.

Approach:

- The algorithm utilized a cascade of five, four, three, two, and one-gram models with a weighted average approach. The choice of five as the cut-off of the n-gram is based on the observation that as the order of n-grams goes beyond five, the number of unique n-grams encountered in the corpus tends to decrease significantly. Choice of the 5-gram model as the cut-off model was to balance the accuracy of capturing longer-range dependencies and the practical limitations imposed by data sparsity.

- After constructing the n-grams, for each n-gram, the algorithm estimates the probability of the next letter given the context of the partially revealed word.
The algorithm then assigns weights to each of the n-gram models based on their importance in guessing the letter. This is followed by the multiplication of the probabilities of the n-gram with their respective weights.

- Based on the calculated weighted average probabilities, the algorithm chooses the letter with the highest probability as the next likely occurring letter.

- If no letter is chosen from the above procedure, the algorithm chooses the words from the most common occurring in the corpus with extra weight given to the vowels.


In [ ]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from collections import defaultdict

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        #array capturing guessed letters
        self.guessed_letters = []
        #array capturing wrong guesses
        self.wrong_guesses= []

        full_dictionary_location = "/content/drive/MyDrive/DATASET/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.unique_alphabets = sorted(set("".join(self.full_dictionary)))
        self.most_freq_alphabets = ['a','e','i','o','u']
        self.prior = [0]*len(self.unique_alphabets)
        self.unigram, self.bigram, self.trigram, self.fourgram, self.fivegram= self.train_ngram()
        self.current_dictionary = []
        #hyperparameters: weights given to ecah of the 5-gram probabilities
        self.weight5 = 0.4
        self.weight4 = 0.25
        self.weight3 = 0.2
        self.weight2 = 0.1
        self.weight1 = 0.05

        #self.weight5 = 0.25
        #self.weight4 = 0.25
        #self.weight3 = 0.42
        #self.weight2 =0.08
        #self.weight1 =1

        #self.weight5 = 0.03864019512995809
        #self.weight4 = 0.15428121558276808
        #self.weight3 = 0.12841833151811646
        #self.weight2 = 0.6786602577691578
        #self.weight1 = 1

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def train_ngram(self):
      unigram = defaultdict(lambda : defaultdict(int))
      bigram =  defaultdict(lambda : defaultdict(lambda : defaultdict(int)))
      trigram = defaultdict(lambda : defaultdict(lambda : defaultdict(int)))
      fourgram = defaultdict(lambda : defaultdict (lambda : defaultdict(lambda : defaultdict(int))))
      fivegram = defaultdict(lambda : defaultdict(lambda : defaultdict (lambda : defaultdict(lambda : defaultdict(int)))))

      for word in self.full_dictionary:
          for i in range(len(word)-4):
            bigram[len(word)][word[i]][word[i+1]]+=1
            trigram[word[i]][word[i+1]][word[i+2]]+=1
            fourgram[word[i]][word[i+1]][word[i+2]][word[i+3]]+=1
            fivegram[word[i]][word[i+1]][word[i+2]][word[i+3]][word[i+4]]+=1
          i =  len(word) - 4

          if len(word)==2:
            bigram[len(word)][word[0]][word[1]]+=1
          elif len(word)==3:
            bigram[len(word)][word[0]][word[1]]+=1
            bigram[len(word)][word[1]][word[2]]+=1
            trigram[word[0]][word[1]][word[2]]+=1
          elif len(word)>=4:
            bigram[len(word)][word[i]][word[i+1]]+=1
            trigram[word[i]][word[i+1]][word[i+2]]+=1
            fourgram[word[i]][word[i+1]][word[i+2]][word[i+3]]+=1
          for alphabet in set(word):
            unigram[len(word)][alphabet]+=1
      return unigram, bigram, trigram, fourgram, fivegram

    def fivegram_probs(self,word):
        posterior = [0]*len(self.unique_alphabets)
        total_combinations = 0
        alphabet_count = [0]*len(self.unique_alphabets)

        print(self.fivegram['a']['p']['p']['l']['e'])
        for i in range(len(word)-4):

          #alphabet alphabet alphabet alphabet blank
          if word[i]!='_' and word[i+1]!='_' and word[i+2]!='_' and word[i+3]!='_' and word[i+4]=='_':
            known_alphabet1 = word[i]
            known_alphabet2 = word[i+1]
            known_alphabet3 = word[i+2]
            known_alphabet4 = word[i+3]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.fivegram[known_alphabet1][known_alphabet2][known_alphabet3][known_alphabet4][probable_alphabet]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.fivegram[known_alphabet1][known_alphabet2][known_alphabet3][known_alphabet4][probable_alphabet]
                alphabet_count[j]+=  self.fivegram[known_alphabet1][known_alphabet2][known_alphabet3][known_alphabet4][probable_alphabet]

          # alphabet alphabet alphabet blank alphabet
          if word[i]!='_' and word[i+1]!='_' and word[i+2]!='_' and word[i+3]=='_' and word[i+4]!='_':
            known_alphabet1 = word[i]
            known_alphabet2 = word[i+1]
            known_alphabet3 = word[i+2]
            known_alphabet4 = word[i+4]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.fivegram[known_alphabet1][known_alphabet2][known_alphabet3][probable_alphabet][known_alphabet4]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.fivegram[known_alphabet1][known_alphabet2][known_alphabet3][probable_alphabet][known_alphabet4]
                alphabet_count[j]+=  self.fivegram[known_alphabet1][known_alphabet2][known_alphabet3][probable_alphabet][known_alphabet4]

          # alphabet alphabet blank alphabet alphabet
          if word[i]!='_' and word[i+1]!='_' and word[i+2]=='_' and word[i+3]!='_' and word[i+4]!='_':
            known_alphabet1 = word[i]
            known_alphabet2 = word[i+1]
            known_alphabet3 = word[i+3]
            known_alphabet4 = word[i+4]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.fivegram[known_alphabet1][known_alphabet2][probable_alphabet][known_alphabet3][known_alphabet4]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.fivegram[known_alphabet1][known_alphabet2][probable_alphabet][known_alphabet3][known_alphabet4]
                alphabet_count[j]+=  self.fivegram[known_alphabet1][known_alphabet2][probable_alphabet][known_alphabet3][known_alphabet4]

          # alphabet blank alphabet alphabet alphabet
          if word[i]!='_' and word[i+1]=='_' and word[i+2]!='_' and word[i+3]!='_' and word[i+4]!='_':
            known_alphabet1 = word[i]
            known_alphabet2 = word[i+2]
            known_alphabet3 = word[i+3]
            known_alphabet4 = word[i+4]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.fivegram[known_alphabet1][probable_alphabet][known_alphabet2][known_alphabet3][known_alphabet4]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.fivegram[known_alphabet1][probable_alphabet][known_alphabet2][known_alphabet3][known_alphabet4]
                alphabet_count[j]+= self.fivegram[known_alphabet1][probable_alphabet][known_alphabet2][known_alphabet3][known_alphabet4]

          # alphabet blank alphabet alphabet alphabet
          if word[i]=='_' and word[i+1]!='_' and word[i+2]!='_' and word[i+3]!='_' and word[i+4]!='_':
            known_alphabet1 = word[i+1]
            known_alphabet2 = word[i+2]
            known_alphabet3 = word[i+3]
            known_alphabet4 = word[i+4]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.fivegram[probable_alphabet][known_alphabet1][known_alphabet2][known_alphabet3][known_alphabet4]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.fivegram[probable_alphabet][known_alphabet1][known_alphabet2][known_alphabet3][known_alphabet4]
                alphabet_count[j]+= self.fivegram[probable_alphabet][known_alphabet1][known_alphabet2][known_alphabet3][known_alphabet4]
        # calculate the probabilities of each letter appearing
        if total_combinations > 0:
                  for i in range(len(self.unique_alphabets)):
                      posterior[i] = alphabet_count[i] / total_combinations

        # interpolate probabilities
        for i, p in enumerate(self.prior):
                  self.prior[i] = p + posterior[i] * self.weight5

        return self.fourgram_probs(word)

    def fourgram_probs(self,word):
        posterior = [0]*len(self.unique_alphabets)
        total_combinations = 0
        alphabet_count = [0]*len(self.unique_alphabets)
        for i in range(len(word)-3):

          #alphabet alphabet alphabet blank
          if word[i]!='_' and word[i+1]!='_' and word[i+2]!='_' and word[i+3]=='_' :
            known_alphabet1 = word[i]
            known_alphabet2 = word[i+1]
            known_alphabet3 = word[i+2]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.fourgram[known_alphabet1][known_alphabet2][known_alphabet3][probable_alphabet]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.fourgram[known_alphabet1][known_alphabet2][known_alphabet3][probable_alphabet]
                alphabet_count[j]+=  self.fourgram[known_alphabet1][known_alphabet2][known_alphabet3][probable_alphabet]

          # alphabet alphabet blank alphabet
          if word[i]!='_' and word[i+1]!='_' and word[i+2]=='_' and word[i+3]!='_':
            known_alphabet1 = word[i]
            known_alphabet2 = word[i+1]
            known_alphabet3 = word[i+3]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.fourgram[known_alphabet1][known_alphabet2][probable_alphabet][known_alphabet3]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.fourgram[known_alphabet1][known_alphabet2][probable_alphabet][known_alphabet3]
                alphabet_count[j]+=  self.fourgram[known_alphabet1][known_alphabet2][probable_alphabet][known_alphabet3]

          # alphabet blank alphabet alphabet
          if word[i]!='_' and word[i+1]=='_' and word[i+2]!='_' and word[i+3]!='_':
            known_alphabet1 = word[i]
            known_alphabet2 = word[i+2]
            known_alphabet3 = word[i+3]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.fourgram[known_alphabet1][probable_alphabet][known_alphabet2][known_alphabet3]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.fourgram[known_alphabet1][probable_alphabet][known_alphabet2][known_alphabet3]
                alphabet_count[j]+=  self.fourgram[known_alphabet1][probable_alphabet][known_alphabet2][known_alphabet3]

          # blank alphabet alphabet alphabet
          if word[i]=='_' and word[i+1]!='_' and word[i+2]!='_' and word[i+3]!='_':
            known_alphabet1 = word[i+1]
            known_alphabet2 = word[i+2]
            known_alphabet3 = word[i+3]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.fourgram[probable_alphabet][known_alphabet1][known_alphabet2][known_alphabet3]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.fourgram[probable_alphabet][known_alphabet1][known_alphabet2][known_alphabet3]
                alphabet_count[j]+= self.fourgram[probable_alphabet][known_alphabet1][known_alphabet2][known_alphabet3]

        # calculate the probabilities of each letter appearing
        if total_combinations > 0:
                for i in range(len(self.unique_alphabets)):
                    posterior[i] = alphabet_count[i] / total_combinations

        # interpolate probabilities
        for i, p in enumerate(self.prior):
                self.prior[i] = p + posterior[i] * self.weight4

        return self.trigram_probs(word)


    def trigram_probs(self,word):
        posterior = [0]*len(self.unique_alphabets)
        total_combinations = 0
        alphabet_count = [0]*len(self.unique_alphabets)
        for i in range(len(word)-2):

          #alphabet alphabet blank
          if word[i]!='_' and word[i+1]!='_' and word[i+2]=='_'  :
            known_alphabet1 = word[i]
            known_alphabet2 = word[i+1]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.trigram[known_alphabet1][known_alphabet2][probable_alphabet]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.trigram[known_alphabet1][known_alphabet2][probable_alphabet]
                alphabet_count[j]+=  self.trigram[known_alphabet1][known_alphabet2][probable_alphabet]

          # alphabet alphabet alphabet blank alphabet
          if word[i]!='_' and word[i+1]=='_' and word[i+2]!='_' :
            known_alphabet1 = word[i]
            known_alphabet2 = word[i+2]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.trigram[known_alphabet1][probable_alphabet][known_alphabet2]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.trigram[known_alphabet1][probable_alphabet][known_alphabet2]
                alphabet_count[j]+=  self.trigram[known_alphabet1][probable_alphabet][known_alphabet2]

          # alphabet alphabet blank alphabet alphabet
          if word[i]=='_' and word[i+1]!='_' and word[i+2]!='_':
            known_alphabet1 = word[i+1]
            known_alphabet2 = word[i+2]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.trigram[probable_alphabet][known_alphabet1][known_alphabet2]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.trigram[probable_alphabet][known_alphabet1][known_alphabet2]
                alphabet_count[j]+=  self.trigram[probable_alphabet][known_alphabet1][known_alphabet2]
        # calculate the probabilities of each letter appearing
        if total_combinations > 0:
                  for i in range(len(self.unique_alphabets)):
                      posterior[i] = alphabet_count[i] / total_combinations

        # interpolate probabilities
        for i, p in enumerate(self.prior):
                  self.prior[i] = p + posterior[i] * self.weight3

        return self.bigram_probs(word)

    def bigram_probs(self, word):
        posterior = [0]*len(self.unique_alphabets)
        total_combinations = 0
        alphabet_count = [0]*len(self.unique_alphabets)
        for i in range(len(word)-3):

          #alphabet blank
          if word[i]!='_' and word[i+1]=='_' :
            known_alphabet = word[i]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.bigram[len(word)][known_alphabet][probable_alphabet]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.bigram[len(word)][known_alphabet][probable_alphabet]
                alphabet_count[j]+=  self.bigram[len(word)][known_alphabet][probable_alphabet]

          # blank alphabet
          if word[i]=='_' and word[i+1]!='_':
            known_alphabet = word[i+1]

            for j, probable_alphabet in enumerate(self.unique_alphabets):
              if self.bigram[len(word)][probable_alphabet][known_alphabet]>0 and probable_alphabet not in self.guessed_letters:
                total_combinations+= self.bigram[len(word)][probable_alphabet][known_alphabet]
                alphabet_count[j]+=  self.bigram[len(word)][probable_alphabet][known_alphabet]

        # calculate the probabilities of each letter appearing
        if total_combinations > 0:
              for i in range(len(self.unique_alphabets)):
                  posterior[i] = alphabet_count[i] / total_combinations

        # interpolate probabilities
        for i, p in enumerate(self.prior):
              self.prior[i] = p + posterior[i] * self.weight2

        return self.unigram_probs(word)

    def unigram_probs(self, word):

        posterior = [0] * len(self.unique_alphabets)

        total_combinations = 0
        alphabet_count = [0] * len(self.unique_alphabets)

        # traverse the word and find blank spaces
        for i in range(len(word)):
            if word[i] == '_':
                for j, probable_alphabet in enumerate(self.unique_alphabets):
                    if self.unigram[len(word)][probable_alphabet] > 0 and probable_alphabet not in self.guessed_letters:
                        total_combinations += self.unigram[len(word)][probable_alphabet]
                        alphabet_count[j] += self.unigram[len(word)][probable_alphabet]

        if total_combinations > 0:
            for i in range(len(self.unique_alphabets)):
                posterior[i] = alphabet_count[i] / total_combinations

        for i, p in enumerate(self.prior):
            self.prior[i] = p + posterior[i] * self.weight1

        # normalizing the probabilities
        sum_all_probs = sum(self.prior)
        if sum_all_probs > 0:
            for i in range(len(self.prior)):
                self.prior[i] = self.prior[i] / sum_all_probs


        # find letter with largest probability
        max_prob = 0
        guess_char = ''
        for i, char in enumerate(self.unique_alphabets):
            if self.prior[i] > max_prob:
                max_prob = self.prior[i]
                guess_char = char
        if guess_char !='':
          return guess_char
        else:
          letters = self.unique_alphabets.copy()
          random.shuffle(letters)
          shuffled_letters = self.most_freq_alphabets+ letters
          for alphabet in self.most_freq_alphabets:
            if alphabet not in self.guessed_letters:
              return alphabet


    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################


        # keep track of incorrect guesses to update the n-grams
        self.incorrect_guesses = list(set(self.guessed_letters) - set(word))
        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2]

        #getting the probabilities to its initial form before making guesses
        self.prior = [0] * len(self.unique_alphabets)

        guess_letter = self.fivegram_probs(clean_word)



        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

In [ ]:
#making a HnagmanAPI class object
api = HangmanAPI()


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [ ]:
for i in range(100):
  api.start_game(practice=1,verbose=True)
  [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
  practice_success_rate = total_practice_successes / total_practice_runs
  print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 1bd39150c449. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ .
50
Guessing letter: i
Sever response: {'game_id': '1bd39150c449', 'status': 'ongoing', 'tries_remains': 6, 'word': 'i _ _ _ _ _ _ _ _ _ _ _ _ _ i _ _ '}
50
Guessing letter: t
Sever response: {'game_id': '1bd39150c449', 'status': 'ongoing', 'tries_remains': 5, 'word': 'i _ _ _ _ _ _ _ _ _ _ _ _ _ i _ _ '}
50
Guessing letter: o
Sever response: {'game_id': '1bd39150c449', 'status': 'ongoing', 'tries_remains': 5, 'word': 'i _ _ _ _ o _ _ _ _ _ _ _ _ i _ _ '}
50
Guessing letter: n
Sever response: {'game_id': '1bd39150c449', 'status': 'ongoing', 'tries_remains': 5, 'word': 'i _ _ _ n o _ _ _ _ _ _ _ _ i _ _ '}
50
Guessing letter: e
Sever response: {'game_id': '1bd39150c449', 'status': 'ongoing', 'tries_remains': 5, 'word': 'i _ _ _ n o _ _ _ _ _ e _ _ i _ e '}
50
Guessing letter: s
Sever response: {'game_id': '1bd39150c449', 'status': 'ongoing', 'tries_remains': 5, 'word':

KeyboardInterrupt: ignored